In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
#last updated by Stefania on the 21/6/18 to include beta star
#last updated by Stefania on the 31/7/18 to include coupling
import numpy as np
#from IBSmodel_GY import IBSmodel
import sys
#from emit_model_elastic_scattering import dedtElastic
from scipy.constants import c

def Lumi_inst(Number_bunches, bunch_intensity_p1, bunch_intensity_p2, ex_norm_m1, ey_norm_m1, ex_norm_m2, ey_norm_m2, bl_4sigma_s1, bl_4sigma_s2, betastar_m, phi_full_rad, gamma, experiment):
    
    sigmaz_m = (bl_4sigma_s1+bl_4sigma_s2)/2./4.*c
    sigmax_m = np.sqrt(ex_norm_m1/gamma*betastar_m+ex_norm_m2/gamma*betastar_m)/np.sqrt(2.)
    sigmay_m = np.sqrt(ey_norm_m1/gamma*betastar_m+ey_norm_m2/gamma*betastar_m)/np.sqrt(2.)

    if not experiment:
        experiment = 'CMS'

    if experiment=='ATLAS':
        FF=1./((1.+(sigmaz_m/sigmay_m*phi_full_rad/2.)**2)**0.5);
    elif experiment=='CMS':
        FF=1./((1.+(sigmaz_m/sigmax_m*phi_full_rad/2.)**2)**0.5);

    return FF*fr*Number_bunches*bunch_intensity_p1*bunch_intensity_p2/(sigmax_m*sigmay_m)/4./np.pi


def LumiModel(gamma=None, betastar_m=None, phi_full_rad_ATLAS=None, phi_full_rad_CMS=None, bunch_intensityin_p1=None, bunch_intensityin_p2=None, exin_norm_m1=None,exin_norm_m2=None, eyin_norm_m1=None, eyin_norm_m2=None, blin_4sigma_s1=None, blin_4sigma_s2=None, tFill_s=None,tauSRxy_s=None, tauSRl_s=None, sigmaBOff_m2=None, sigmaElastic_m2=None, VRF_V=None, IBSON=1, emitBU="Model", BoffON=1, coupling=1, coupling_coeff=0.0015, delta_ts=0.01, noiseGrowth=1, boffGrowth=1, blengthBU='Model', nIPs = 2., dt_s=15*60., tau_empirical_h1=None, tau_empirical_v1=None, tau_empirical_h2=None, tau_empirical_v2=None, leveling=None):

    if not gamma:
                gamma = 6927.64 #6.5TeV
                #    if not betastar_m:
                #	betastar_m = .4
    if not tauSRxy_s:
                tauSRxy_s = 64.7*3600;
                tauSRl_s = 32.35*3600;
    if not sigmaBOff_m2:
                sigmaBOff_m2 = 82*1e-31;
                sigma_el_m2 = 29.7*1e-31;

    if BoffON:
                LossesFromData = 0
    else:
                LossesFromData = 1

   
    ex_norm_m1 = [exin_norm_m1[0,:]]
    ey_norm_m1 = [eyin_norm_m1[0,:]]
    ex_norm_m2 = [exin_norm_m2[0,:]]
    ey_norm_m2 = [eyin_norm_m2[0,:]]
    bl_4sigma_s1 = [blin_4sigma_s1[0,:]]
    bl_4sigma_s2 = [blin_4sigma_s2[0,:]]
    bunch_intensity_p1 = [bunch_intensityin_p1[0,:]]
    bunch_intensity_p2 = [bunch_intensityin_p2[0,:]]
    ex_norm_m1_IBScorr = [exin_norm_m1[0,:]]
    ey_norm_m1_IBScorr = [eyin_norm_m1[0,:]]
    ex_norm_m2_IBScorr = [exin_norm_m2[0,:]]
    ey_norm_m2_IBScorr = [eyin_norm_m2[0,:]]

    if filln > 6400: #2018
        set_betastar_m = betastar_m[0]
    else:            #2017 and older
        set_betastar_m = betastar_m
    
    Lumi_init_ATLAS = Lumi_inst(Number_bunches=1., bunch_intensity_p1=bunch_intensityin_p1[0,:], bunch_intensity_p2=bunch_intensityin_p2[0,:],
                                ex_norm_m1=exin_norm_m1[0,:], ey_norm_m1=eyin_norm_m1[0,:], ex_norm_m2=exin_norm_m2[0,:], ey_norm_m2=eyin_norm_m2[0,:],
                                bl_4sigma_s1=blin_4sigma_s1[0,:], bl_4sigma_s2=blin_4sigma_s2[0,:],
                                betastar_m=set_betastar_m, phi_full_rad=phi_full_rad_ATLAS[0], gamma=gamma, experiment='ATLAS')

    Lumi_init_CMS = Lumi_inst(Number_bunches=1., bunch_intensity_p1=bunch_intensityin_p1[0,:], bunch_intensity_p2=bunch_intensityin_p2[0,:],
                              ex_norm_m1=exin_norm_m1[0,:], ey_norm_m1=eyin_norm_m1[0,:], ex_norm_m2=exin_norm_m2[0,:], ey_norm_m2=eyin_norm_m2[0,:],
                              bl_4sigma_s1=blin_4sigma_s1[0,:], bl_4sigma_s2=blin_4sigma_s2[0,:],
                              betastar_m=set_betastar_m, phi_full_rad=phi_full_rad_CMS[0], gamma=gamma, experiment='CMS')

    if leveling:
                Lumi_init_ATLAS = leveling/len(Lumi_init_ATLAS)*np.ones(np.shape(Lumi_init_ATLAS))
                Lumi_init_CMS = leveling/len(Lumi_init_CMS)*np.ones(np.shape(Lumi_init_CMS))

    Luminosity_ATLAS_invm2s = [Lumi_init_ATLAS]
    Luminosity_CMS_invm2s = [Lumi_init_CMS]
    Luminosity_invm2s = [(Lumi_init_ATLAS + Lumi_init_CMS)/2.]
    tt_s = np.arange(0., tFill_s, dt_s)
    N_steps = len(tt_s)

    dex_noise1=[];dey_noise1=[];dex_noise2=[];dey_noise2=[]
    deltaBPMnow_hb1=[];deltaBPMnow_vb1=[];deltaBPMnow_hb2=[];deltaBPMnow_vb2=[]; xi_hb1=[];xi_vb1=[];xi_hb2=[];xi_vb2=[]; 
    de_boff1h=[];de_boff1v=[];de_boff2h=[];de_boff2v=[];
    
    bunch_intensity_p1_boff=bunch_intensity_p1
    bunch_intensity_p2_boff=bunch_intensity_p2
    for i_step in xrange(1, N_steps):
        print i_step
        # Applies the IBS + SR model
        ex_IBS_norm_m1, IBSx1, bl_IBS_4sigma_s1, IBSl1, ey_IBS_norm_m1 = IBSmodel(IBSON, gamma,bunch_intensity_p1[i_step-1],ex_norm_m1[i_step-1],ey_norm_m1[i_step-1],bl_4sigma_s1[i_step-1],VRF_V,dt_s)
        ex_IBS_norm_m2, IBSx2, bl_IBS_4sigma_s2, IBSl2, ey_IBS_norm_m2 = IBSmodel(IBSON, gamma,bunch_intensity_p2[i_step-1],ex_norm_m2[i_step-1],ey_norm_m2[i_step-1],bl_4sigma_s2[i_step-1],VRF_V,dt_s)
        ey_IBS_norm_m1 = ey_norm_m1[i_step-1]*np.exp(-2*dt_s/tauSRxy_s)
        ey_IBS_norm_m2 = ey_norm_m2[i_step-1]*np.exp(-2*dt_s/tauSRxy_s)

        # Elastic scattering part
        if filln > 6400: #2018
            set_betastar_mm = betastar_m[i_step-1]
        else:            #2017 and older
            set_betastar_mm = betastar_m
        
        dexdt1, deconvdt1 = dedtElastic(2, np.array(bunch_intensity_p1[i_step-1]), 1, np.array(Luminosity_invm2s[i_step-1]), set_betastar_mm, sigmaElastic_m2)
        dexdt2, deconvdt2 = dedtElastic(2, np.array(bunch_intensity_p2[i_step-1]), 1, np.array(Luminosity_invm2s[i_step-1]), set_betastar_mm, sigmaElastic_m2)
        
        #emittance growth due to coupling
        #Coupling_coeff_FT            = 0.0015    # Linear coupling coefficient for Model in FT
        #Delta_tunesplit_FT           = 0.01      # Unperturbed tune split in FT    
                
        if coupling:
            coupling_coeff=Coupling_coeff_FT
            elta_ts=Delta_tunesplit_FT
                        
            Cfactor = coupling_coeff**2/2/(delta_ts**2+coupling_coeff**2)
            dexy1   = ex_IBS_norm_m1-ey_IBS_norm_m1
            dexy2   = ex_IBS_norm_m2-ey_IBS_norm_m2
            #print Cfactor,dexy1,dexy2
        else:
            Cfactor = 0
            dexy1   = 0
            dexy2   = 0
          
        #ex_IBS_norm_m1 = ex_IBS_norm_m1  -  dexy1 * Cfactor
        #ey_IBS_norm_m1 = ey_IBS_norm_m1  +  dexy1 * Cfactor
        #ex_IBS_norm_m2 = ex_IBS_norm_m2  -  dexy2 * Cfactor
        #ey_IBS_norm_m2 = ey_IBS_norm_m2  +  dexy2 * Cfactor 
                           
        #emittance growth due to noise
        if filln > 6400: #2018
            set_betastar_mmm = betastar_m[i_step]
        else:            #2017 and older
            set_betastar_mmm = betastar_m
            
        getGrowth_hb1,deltaBPMnoww_hb1,xii_hb1= getNoiseGrowth(energy=gamma*0.938, sigs=np.mean(blin_4sigma_s1[i_step]/4.0*c), intensity=np.mean(bunch_intensityin_p1[i_step]), emit=np.mean(ex_IBS_norm_m1), xing=phi_full_rad_ATLAS[i_step], beta=set_betastar_mmm,sigmaElastic_m2=sigmaElastic_m2, delta=delta_hb1, deltaBPM_SB=deltaBPM_SB_hb1)
        getGrowth_vb1,deltaBPMnoww_vb1,xii_vb1= getNoiseGrowth(energy=gamma*0.938, sigs=np.mean(blin_4sigma_s1[i_step]/4.0*c), intensity=np.mean(bunch_intensityin_p1[i_step]), emit=np.mean(ey_IBS_norm_m1), xing=phi_full_rad_ATLAS[i_step], beta=set_betastar_mmm,sigmaElastic_m2=sigmaElastic_m2, delta=delta_vb1, deltaBPM_SB=deltaBPM_SB_vb1)
        getGrowth_hb2,deltaBPMnoww_hb2,xii_hb2= getNoiseGrowth(energy=gamma*0.938, sigs=np.mean(blin_4sigma_s2[i_step]/4.0*c), intensity=np.mean(bunch_intensityin_p2[i_step]), emit=np.mean(ex_IBS_norm_m2), xing=phi_full_rad_ATLAS[i_step], beta=set_betastar_mmm,sigmaElastic_m2=sigmaElastic_m2, delta=delta_hb2, deltaBPM_SB=deltaBPM_SB_hb2)
        getGrowth_vb2,deltaBPMnoww_vb2,xii_vb2= getNoiseGrowth(energy=gamma*0.938, sigs=np.mean(blin_4sigma_s2[i_step]/4.0*c), intensity=np.mean(bunch_intensityin_p2[i_step]), emit=np.mean(ey_IBS_norm_m2), xing=phi_full_rad_ATLAS[i_step], beta=set_betastar_mmm,sigmaElastic_m2=sigmaElastic_m2, delta=delta_vb2, deltaBPM_SB=deltaBPM_SB_vb2)

        if noiseGrowth:#emittance growth due to noise, apply for both colliding and non-colliding
        #the noise growths are assumed to be constant along the fill, because there is the balance between the variation of the intensity and the adt impact                    
        #improve this by giving the input parameters to the noise formula, so that to get the exact growths for each case
        #theNoiseGrowths=na([0.038,0.057,0.036,0.05])                         
            dex_noisee1=ex_IBS_norm_m1*getGrowth_hb1/100.0*(dt_step/60.0) #in percentage per hour, for a 5 min step since the data are saved every 5min
            dey_noisee1=ey_IBS_norm_m1*getGrowth_vb1/100.0*(dt_step/60.0)
            dex_noisee2=ex_IBS_norm_m2*getGrowth_hb2/100.0*(dt_step/60.0)
            dey_noisee2=ey_IBS_norm_m2*getGrowth_vb2/100.0*(dt_step/60.0)        
            #print '~',dex_noise1
            #dex_noise1=0.003166667*1e-06;dey_noise1=0.00475*1e-06;dex_noise2=0.003*1e-06;dey_noise2=0.004166667*1e-06; #the growth in um for 5min
            #ex_IBS_norm_m1 = ex_IBS_norm_m1  +  dex_noisee1
            #ey_IBS_norm_m1 = ey_IBS_norm_m1  +  dey_noisee1
            #ex_IBS_norm_m2 = ex_IBS_norm_m2  +  dex_noisee2
            #ey_IBS_norm_m2 = ey_IBS_norm_m2  +  dey_noisee2    
        else:
            dex_noisee1=0;
            dey_noisee1=0;
            dex_noisee2=0;
            dey_noisee2=0;         
                        
        #print 'hb1->',np.mean(dex_noisee1),deltaBPMnoww_hb1,xii_hb1
        #print 'vb1->',np.mean(dey_noisee1),deltaBPMnoww_vb1,xii_vb1
        #print 'hb2->',np.mean(dex_noisee2),deltaBPMnoww_hb2,xii_hb2
        #print 'vb2->',np.mean(dey_noisee2),deltaBPMnoww_vb2,xii_vb2
        
        #Intensity Variation
        if BoffON==1:
        #~ tauBOff_a = nb*Nb0[i-1]/(nIPs*sigmaBOff*L0[i-1]));#   % in minutes
                tauBOff_s1 = bunch_intensity_p1[i_step-1]/(sigmaBOff_m2*nIPs*(Luminosity_invm2s[i_step-1]))
                bunch_intensity_p1.append(bunch_intensity_p1[i_step-1]/(1.+dt_s/tauBOff_s1));
                tauBOff_s2 = bunch_intensity_p2[i_step-1]/(sigmaBOff_m2*nIPs*Luminosity_invm2s[i_step-1])
                bunch_intensity_p2.append(bunch_intensity_p2[i_step-1]/(1.+dt_s/tauBOff_s2));             
        elif LossesFromData == 1:   
                #losses from data
                bunch_intensity_p1.append(bunch_intensityin_p1[i_step]);
                bunch_intensity_p2.append(bunch_intensityin_p2[i_step]);     
        else:
                bunch_intensity_p1.append(bunch_intensity_p1[i_step]);
                bunch_intensity_p2.append(bunch_intensity_p2[i_step]);     
                
        #emittance growth due to Boff
        if sigmaElastic_m2 != 0:  #so that to take into account the colliding only
            if boffGrowth: #emittance growth due to B0ff -> ~%4 that is emit*4/100*1e6. the growth is 0.000554um for 5min
                #improve this by giving the input parameters, so that to get the exact growths:
                #->de=(0.075/0.20)*(dNBo/NBo)*e and so...
                #multip_factor=0.075/0.20 #around 20% of protons are lost because of Boff, based on slide 25 in https://indico.cern.ch/event/806869/contributions/3358585/attachments/1816649/2969341/Emittance_BU_due_to_BO.pdf
                #we take the burn-off bunch intensity, not the intensity from data
                
                tauBOffs1 = bunch_intensity_p1_boff[i_step-1]/(sigmaBOff_m2*nIPs*(Luminosity_invm2s[i_step-1]))
                bunch_intensity_p1_boff=bunch_intensity_p1_boff[i_step-1]/(1.+dt_s/tauBOffs1);
                tauBOffs2 = bunch_intensity_p2_boff[i_step-1]/(sigmaBOff_m2*nIPs*Luminosity_invm2s[i_step-1])
                bunch_intensity_p2_boff=bunch_intensity_p2_boff[i_step-1]/(1.+dt_s/tauBOffs2);
                
                dNBB1=abs(bunch_intensity_p1_boff[i_step]-bunch_intensity_p1_boff[i_step-1])/bunch_intensity_p1_boff[i_step-1] #(dNBo1/NBo1)
                dNBB2=abs(bunch_intensity_p2_boff[i_step]-bunch_intensity_p2_boff[i_step-1])/bunch_intensity_p2_boff[i_step-1] #(dNBo2/NBo2)
                #print ex_IBS_norm_m1[i_step+1]
                #print i_step, np.size(ex_IBS_norm_m1), np.size(ex_IBS_norm_m1[i_step-1])
                
                if np.mean(dNBB1)>0.01:
                    dNBB1=0
                if np.mean(dNBB2)>0.01:
                    dNBB2=0
                print np.mean(dNBB1),np.mean(dNBB2)
                de_bofff1h=multip_factor*dNBB1*ex_IBS_norm_m1
                de_bofff1v=multip_factor*dNBB1*ey_IBS_norm_m1
                de_bofff2h=multip_factor*dNBB2*ex_IBS_norm_m2
                de_bofff2v=multip_factor*dNBB2*ey_IBS_norm_m2
                        
                #ex_IBS_norm_m1 = ex_IBS_norm_m1  +  de_bofff1h
                #ey_IBS_norm_m1 = ey_IBS_norm_m1  +  de_bofff1v
                #ex_IBS_norm_m2 = ex_IBS_norm_m2  +  de_bofff2h
                #ey_IBS_norm_m2 = ey_IBS_norm_m2  +  de_bofff2v   
                        
            else:
                de_bofff1h=0
                de_bofff1v=0
                de_bofff2h=0
                de_bofff2v=0 

        else:
            de_bofff1h=0
            de_bofff1v=0
            de_bofff2h=0
            de_bofff2v=0  

        #print  '--->',np.mean(ex_IBS_norm_m1), np.mean(ex_IBS_norm_m1-dexy1*Cfactor)           
        #print  '--->',np.mean(ex_IBS_norm_m1)#, np.mean(ex_IBS_norm_m1-dexy1*Cfactor)   
        if emitBU=='Model':
            ex_norm_m1.append(ex_IBS_norm_m1+dexdt1*dt_s -dexy1*Cfactor +dex_noisee1 +de_bofff1h)
            ey_norm_m1.append(ey_IBS_norm_m1+dexdt1*dt_s +dexy1*Cfactor +dey_noisee1 +de_bofff1v)
            ex_norm_m2.append(ex_IBS_norm_m2+dexdt2*dt_s -dexy2*Cfactor +dex_noisee2 +de_bofff2h)
            ey_norm_m2.append(ey_IBS_norm_m2+dexdt2*dt_s +dexy2*Cfactor +dey_noisee2 +de_bofff2v)
        elif emitBU=='EmpiricalBlowup':
        #ex_norm_m1.append(ex_norm_m1[i_step-1]*np.exp(dt_s/tau_empirical_h1))
        #ey_norm_m1.append(ey_norm_m1[i_step-1]*np.exp(dt_s/tau_empirical_v1))
        #ex_norm_m2.append(ex_norm_m2[i_step-1]*np.exp(dt_s/tau_empirical_h2))
        #ey_norm_m2.append(ey_norm_m2[i_step-1]*np.exp(dt_s/tau_empirical_v2))
            ex_norm_m1.append(exin_norm_m1[i_step])
            ey_norm_m1.append(eyin_norm_m1[i_step])
            ex_norm_m2.append(exin_norm_m2[i_step])
            ey_norm_m2.append(eyin_norm_m2[i_step])

        else:
            raise ValueError('Not understood!')
        #print np.mean(de_bofff1h)
        # Emittance evolution corrected from IBS and elastic scattering for each point in time
        ex_norm_m1_IBScorr.append(ex_IBS_norm_m1+dexdt1*dt_s -dexy1*Cfactor +dex_noisee1 +de_bofff1h)
        ey_norm_m1_IBScorr.append(ey_IBS_norm_m1+dexdt1*dt_s +dexy1*Cfactor +dey_noisee1 +de_bofff1v)
        ex_norm_m2_IBScorr.append(ex_IBS_norm_m2+dexdt2*dt_s -dexy2*Cfactor +dex_noisee2 +de_bofff2h)
        ey_norm_m2_IBScorr.append(ey_IBS_norm_m2+dexdt2*dt_s +dexy2*Cfactor +dey_noisee2 +de_bofff2v)
        
        #print bl_IBS_4sigma_s1
        if blengthBU=='Model':
                bl_4sigma_s1.append(bl_IBS_4sigma_s1)
                bl_4sigma_s2.append(bl_IBS_4sigma_s2)
        elif blengthBU=='EmpiricalBlowup':
                bl_4sigma_s1.append(blin_4sigma_s1[i_step])
                bl_4sigma_s2.append(blin_4sigma_s2[i_step])
        #print dex_noise1 
                                                             
        #noise growth parameters
        dex_noise1.append(dex_noisee1)
        dey_noise1.append(dey_noisee1)
        dex_noise2.append(dex_noisee2)
        dey_noise2.append(dey_noisee2)
        deltaBPMnow_hb1.append(deltaBPMnoww_hb1)
        deltaBPMnow_vb1.append(deltaBPMnoww_vb1)
        deltaBPMnow_hb2.append(deltaBPMnoww_hb2)
        deltaBPMnow_vb2.append(deltaBPMnoww_vb2)
        xi_hb1.append(xii_hb1)
        xi_vb1.append(xii_vb1)
        xi_hb2.append(xii_hb2)
        xi_vb2.append(xii_vb2) 
                
        #boff growth parameters
        de_boff1h.append(de_bofff1h)
        de_boff1v.append(de_bofff1v)
        de_boff2h.append(de_bofff2h)
        de_boff2v.append(de_bofff2v)                                
        
        Lumi_ATLAS = Lumi_inst(Number_bunches=1., bunch_intensity_p1=bunch_intensity_p1[i_step], bunch_intensity_p2=bunch_intensity_p2[i_step], ex_norm_m1=ex_norm_m1[i_step], ey_norm_m1=ey_norm_m1[i_step], ex_norm_m2=ex_norm_m2[i_step], ey_norm_m2=ey_norm_m2[i_step], bl_4sigma_s1=bl_4sigma_s1[i_step], bl_4sigma_s2=bl_4sigma_s2[i_step], betastar_m=set_betastar_mmm, phi_full_rad=phi_full_rad_ATLAS[i_step], gamma=gamma, experiment='ATLAS')
        Lumi_CMS = Lumi_inst(Number_bunches=1., bunch_intensity_p1=bunch_intensity_p1[i_step], bunch_intensity_p2=bunch_intensity_p2[i_step], ex_norm_m1=ex_norm_m1[i_step], ey_norm_m1=ey_norm_m1[i_step], ex_norm_m2=ex_norm_m2[i_step], ey_norm_m2=ey_norm_m2[i_step], bl_4sigma_s1=bl_4sigma_s1[i_step], bl_4sigma_s2=bl_4sigma_s2[i_step], betastar_m=set_betastar_mmm, phi_full_rad=phi_full_rad_CMS[i_step], gamma=gamma, experiment='CMS')

        if leveling and np.sum(Lumi_ATLAS)>leveling:
            Lumi_ATLAS = leveling/len(Lumi_ATLAS)*np.ones(np.shape(Lumi_ATLAS))
        if leveling and np.sum(Lumi_CMS)>leveling:
            Lumi_CMS = leveling/len(Lumi_CMS)*np.ones(np.shape(Lumi_CMS))

        Luminosity_invm2s.append((Lumi_ATLAS+Lumi_CMS)/2.)
        Luminosity_ATLAS_invm2s.append(Lumi_ATLAS)
        Luminosity_CMS_invm2s.append(Lumi_CMS)

        
    return tt_s, Luminosity_ATLAS_invm2s, Luminosity_CMS_invm2s, bunch_intensity_p1, bunch_intensity_p2, ex_norm_m1, ex_norm_m2, ey_norm_m1, ey_norm_m2, bl_4sigma_s1, bl_4sigma_s2, ex_norm_m1_IBScorr, ex_norm_m2_IBScorr, ey_norm_m1_IBScorr, ey_norm_m2_IBScorr, dex_noise1,dey_noise1,dex_noise2,dey_noise2, deltaBPMnow_hb1,deltaBPMnow_vb1,deltaBPMnow_hb2,deltaBPMnow_vb2, xi_hb1,xi_vb1,xi_hb2,xi_vb2, de_boff1h,de_boff1v,de_boff2h,de_boff2v